In [ ]:
# run following commands when this script is executed in google colab
# takes care of cloning the repository and changing the working directory to the repository so objects can be imported from src folder in the repository.
!git clone https://github.com/meeslindhout/Master-Thesis-Project---Cold-Start-Recommender-System-Session-based.git
import os
os.chdir('Master-Thesis-Project---Cold-Start-Recommender-System-Session-based')

# Item knn sesion based recommendation
This is the base model that will be used to compare the RL model with.

In [ ]:
import pandas as pd
from recsys_baseline import ContextKNN
from utils import ensure_dir, save_model, write_results_csv, eval_algorithm